# 1. Import and Install Dependencies

In [ ]:
pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
pip install scikit-learn

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

#this camera in below for show and make sure algoritm work correctly

In [ ]:
output_directory = 'D:\\Data_Set'  
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Iterate through all items in the output directory
    for item in os.listdir(output_directory):
        # Get the full path of the item
        item_path = os.path.join(output_directory, item)
        
        # Check if it's a directory
        if os.path.isdir(item_path):
            for filename in os.listdir(item_path):
                # Get the full path of the file
                file_path = os.path.join(item_path, filename)

                # Check if it's a file
                if os.path.isfile(file_path):
                    frame = cv2.imread(file_path)

                    # Check if the frame was successfully read
                    if frame is not None:
                        # Make detections & Draw landmarks
                        image, results = mediapipe_detection(frame, holistic)
                        print(results)
                        draw_landmarks(image, results)
                        
                        cv2.waitKey(0)

    cv2.destroyAllWindows()


In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
len(results.pose_landmarks.landmark)

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
len(results.pose_landmarks.landmark)

In [ ]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
np.save('Result', result_test)

In [ ]:
np.load('Result.npy')

# 4. Setup Folders for Collection

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('data_mp') 

# Actions that we try to detect
actions = np.array(['Like' , 'Ok' , 'Stop'])
for action in actions: 
        try: 
            os.makedirs(os.path.join(DATA_PATH, action))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
# Initialize MediaPipe Holistic model
mp_holistic = mp.solutions.holistic

# Update the output directory path
output_directory = r'D:\Data_Set'

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        # Create action folder if it doesn't exist
        action_folder = os.path.join(output_directory, action)
        os.makedirs(action_folder, exist_ok=True)
        
        # Loop through folders in the output directory
        for folder_name in os.listdir(output_directory):
            folder_path = os.path.join(output_directory, folder_name)
            output = os.path.join(DATA_PATH, folder_name)
            # Check if the item is a directory
            if os.path.isdir(folder_path):
                counter = 0
                # Loop through files in the folder
                for frame_file in os.listdir(folder_path):
                    frame_path = os.path.join(folder_path, frame_file)
                    
                    if frame_file.endswith(('.jpg', '.jpeg', '.png')):
                        frame = cv2.imread(frame_path)
                        # Ensure that the frame is not None (i.e., image reading is successful)
                        if frame is not None:
                            # Make detections using MediaPipe Holistic & Draw landmarks &  Export keypoints
                            image, results = mediapipe_detection(frame, holistic)
                            draw_styled_landmarks(image, results)
                            keypoints = extract_keypoints(results)
                            
                            # Save keypoints as .npy file in the action folder
                            npy_filename = f"{counter}.npy"
                            npy_filename = os.path.splitext(frame_file)[0] + '.npy'
                            npy_path = os.path.join(output, npy_filename)
                            np.save(npy_path, keypoints)
                            counter+=1
                        else:
                            print(f"Failed to read image: {frame_path}")

print("Conversion completed successfully!")


In [ ]:

cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
import re

sequences, labels = [], []
for action in actions:
    for sequence_file in os.listdir(os.path.join(DATA_PATH, action)):
        # Use regular expression to extract numeric part of the file name
        match = re.match(r'(\d+)', sequence_file)
        if match:
            sequence_number = int(match.group(1))  # Extract the numeric part
            window = []
            res = np.load(os.path.join(DATA_PATH, action, sequence_file))
            window.append(res)
            sequences.append(window)
            labels.append(label_map[action])


In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
X_train.shape

# 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', name='LSTM1', input_shape=(1,126)))
model.add(LSTM(128, return_sequences=True, activation='relu', name='LSTM2'))
model.add(LSTM(64, return_sequences=False, activation='relu', name='LSTM3'))
model.add(Dense(64, activation='relu', name='Dense1'))
model.add(Dense(32, activation='relu',name='Dense2'))
model.add(Dense(actions.shape[0], activation='softmax',name='Dense3'))

In [ ]:
actions.shape[0]

In [ ]:
res = [0.2,0.7,0.1]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

In [ ]:
model.summary()

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[6])]

# 9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# 11. Test in Real Time

In [ ]:
from scipy import stats

In [ ]:
# 1. New detection variables
sequence = []  
sentence = []  
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read frame
        ret, frame = cap.read()

        # Make detections & Draw landmarks 
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-1:]
        
        if len(sequence) == 1:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        # Viz logic
        if np.unique(predictions[-10:])[0] == np.argmax(res):
          if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                 if actions[np.argmax(res)] != sentence[-1]:
                   sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
 
        if len(sentence) > 5:
          sentence = sentence[-5:]

        # Displaying only one element from the sentence
        sentence_to_display = sentence[-1] if len(sentence) > 0 else ""
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, sentence_to_display, (3,30), 
               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()